In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1Metal device set to: Apple M1 Pro
 Physical GPUs, 1 Logical GPUs


In [4]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))
print("Num CPUs Available: ", tf.config.list_physical_devices('CPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num CPUs Available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [5]:
# Load and tokenize text
with open('big_en.txt', 'r') as f:
    text = f.read()
    
len(text)

6488665

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Tokenize the text
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Generate sequences and their corresponding outputs
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = 10
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
X, labels = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)


In [7]:
# Build the MLP model
model = Sequential()
model.add(Embedding(total_words, 32, input_length=max_sequence_len-1))  # Embedding layer
model.add(Flatten())  # Flatten layer to flatten the output of the embedding layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(total_words, activation='softmax'))  # Output layer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9, 32)             1011840   
                                                                 
 flatten (Flatten)           (None, 288)               0         
                                                                 
 dense (Dense)               (None, 32)                9248      
                                                                 
 dense_1 (Dense)             (None, 31620)             1043460   
                                                                 
Total params: 2,064,548
Trainable params: 2,064,548
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the model
with tf.device('/CPU:0'):
    model.fit(X, y, epochs = 3, verbose=1)

# Function to predict the next word
def predict_next_word(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = tokenizer.index_word[predicted[0]]
    return output_word

# Example usage
seed_text = "the"
predicted_word = predict_next_word(seed_text)
print("Next word after '{}' is '{}'".format(seed_text, predicted_word))

Epoch 1/3


2023-05-15 18:32:14.398650: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


22288/22288 [==============================] - 265s 12ms/step - loss: 4.8190 - accuracy: 0.1608
Epoch 2/3
 2891/22288 [==>...........................] - ETA: 3:51 - loss: 4.4965 - accuracy: 0.1843